# 第六章

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip

--2020-04-16 05:52:43--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  27.5MB/s    in 1.0s    

2020-04-16 05:52:44 (27.5 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]



In [0]:
!unzip NewsAggregatorDataset.zip

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [0]:
import pandas as pd

## 50

In [0]:
columns = ["id", "title", "url", "publisher", "category", "story", "hostname", "timestamp"]
dataset_raw = pd.read_table("newsCorpora.csv", names=columns)

In [0]:
search = ["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"]

In [0]:
dataset = dataset_raw[dataset_raw["publisher"].isin(search)]

In [0]:
dataset = dataset.replace({'b': 'Business', 'e': 'Entertainment', 'm': 'Health', 't': 'Technology'})

In [0]:
shuffled = dataset.sample(frac=1).reset_index(drop=True)

In [0]:
import math

In [0]:
train = shuffled[:math.floor(shuffled.shape[0]*0.8)][["category", "title"]]
valid = shuffled[math.floor(shuffled.shape[0]*0.8):math.floor(shuffled.shape[0]*0.9)][["category", "title"]]
test = shuffled[math.floor(shuffled.shape[0]*0.9):][["category", "title"]]

In [0]:
train[["category", "title"]].to_csv("train.csv", sep="\t", index=None, columns=None)
valid[["category", "title"]].to_csv("valid.csv", sep="\t", index=None, columns=None)
test[["category", "title"]].to_csv("test.csv", sep="\t", index=None, columns=None)

In [87]:
print(train.shape[0], valid.shape[0], test.shape[0])

10672 1334 1334


## 51～

In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

とりあえずTf-Idfで10000次元のやつ作ってそのままぶち込んだ

In [0]:
v = TfidfVectorizer(token_pattern=u'(?u)\\b\\w+\\b')

In [0]:
vects = v.fit_transform(train['title'].to_numpy())

In [0]:
v.get_feature_names()

In [0]:
X = vects

In [0]:
Y = train['category'].to_numpy()

In [0]:
logreg = LogisticRegression(max_iter=100000)

In [93]:
logreg.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
test_X = v.transform(test['title'].to_numpy())

In [0]:
test_Y = test['category'].to_numpy()

In [0]:
pred_Y = logreg.predict(test_X)

## 54.スコア

In [97]:
logreg.score(test_X, test_Y)

0.9002998500749625

In [98]:
logreg.score(X, Y)

0.9447151424287856

## 55.混同行列

In [99]:
confusion_matrix(test_Y, pred_Y)

array([[527,  26,   0,   9],
       [  5, 547,   0,   1],
       [ 16,  23,  44,   0],
       [ 27,  26,   0,  83]])

## 56.適合率precision、再現率recall、F1スコア

- 適合率
  システムが出した結果において、本当に正しかったものの割合

- 再現率 recall
  本当に陽性であるケースのうち、何%を陽性と判断できたかの割合

- F1スコア
  それらの調和平均

In [100]:
print(classification_report(test_Y, pred_Y))

               precision    recall  f1-score   support

     Business       0.92      0.94      0.93       562
Entertainment       0.88      0.99      0.93       553
       Health       1.00      0.53      0.69        83
   Technology       0.89      0.61      0.72       136

     accuracy                           0.90      1334
    macro avg       0.92      0.77      0.82      1334
 weighted avg       0.90      0.90      0.89      1334



macro avg ... マクロ平均

weighted avg ... マイクロ平均？
（着目する粒度がマクロかマイクロかってことかな？）

## 57.重み

In [101]:
logreg.intercept_

array([ 0.57474419,  0.90060995, -0.95189077, -0.52346337])

In [0]:
coef = logreg.coef_

In [0]:
coef_rank = np.argsort(coef)

In [0]:
fnames = v.get_feature_names()

In [113]:
for rank, cate in zip(coef_rank, ["Business", "Entertainment", "Health", "Technology"]):
  rank_label_b = [fnames[i] for i in rank[::-1][:10]]
  rank_label_w = [fnames[i] for i in rank[:10]]
  print(cate)
  print("Best:", " ".join(rank_label_b))
  print("Worst:", " ".join(rank_label_w))

Business
Best: fed bank china stocks ecb euro oil ukraine dollar profit
Worst: the ebola and she her apple google study drug facebook
Entertainment
Best: kardashian chris she kim her miley cyrus movie star paul
Worst: us google 1 update china gm facebook says study ceo
Health
Best: ebola cancer study drug fda mers health heart cases outbreak
Worst: gm facebook s apple google ceo climate sales twitter amazon
Technology
Best: google facebook apple climate microsoft gm tesla nasa fcc comcast
Worst: stocks fed drug ecb cancer american kardashian ebola her ukraine


## 58.正則化パラメータの変更

In [0]:
logreg_c = LogisticRegression(max_iter=100000, C=10)

In [115]:
logreg_c.fit(X, Y)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
pred_Y = logreg_c.predict(test_X)

In [117]:
print(classification_report(test_Y, pred_Y))

               precision    recall  f1-score   support

     Business       0.94      0.94      0.94       562
Entertainment       0.93      0.98      0.95       553
       Health       0.95      0.70      0.81        83
   Technology       0.85      0.74      0.79       136

     accuracy                           0.92      1334
    macro avg       0.92      0.84      0.87      1334
 weighted avg       0.92      0.92      0.92      1334



ちょい上昇

In [0]:
logreg_c = LogisticRegression(max_iter=100000, C=100)

In [119]:
logreg_c.fit(X, Y)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
pred_Y = logreg_c.predict(test_X)

In [121]:
print(classification_report(test_Y, pred_Y))

               precision    recall  f1-score   support

     Business       0.94      0.94      0.94       562
Entertainment       0.93      0.98      0.96       553
       Health       0.94      0.73      0.82        83
   Technology       0.83      0.78      0.80       136

     accuracy                           0.93      1334
    macro avg       0.91      0.86      0.88      1334
 weighted avg       0.93      0.93      0.93      1334



Healthに関する過学習がなくなってきている気がする

In [122]:
c = 1000

logreg_c = LogisticRegression(max_iter=100000, C=c)
logreg_c.fit(X, Y)
pred_Y = logreg_c.predict(test_X)
print(classification_report(test_Y, pred_Y))

               precision    recall  f1-score   support

     Business       0.94      0.95      0.95       562
Entertainment       0.94      0.98      0.96       553
       Health       0.94      0.76      0.84        83
   Technology       0.83      0.79      0.81       136

     accuracy                           0.93      1334
    macro avg       0.91      0.87      0.89      1334
 weighted avg       0.93      0.93      0.93      1334



同じく

## 59.ハイパーパラメータ探索

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
logreg_cv = GridSearchCV(
    LogisticRegression(max_iter=100000),
    {
      'C': [1, 10, 100, 1000],
      'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    },
    cv=3)

In [128]:
logreg_cv.fit(X, Y)

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10, 100, 1000],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [0]:
logreg_cv.cv_results_

In [130]:
logreg_cv.best_params_

{'C': 100, 'solver': 'liblinear'}

In [0]:
logreg_best = logreg_cv.best_estimator_

In [0]:
pred_Y = logreg_best.predict(test_X)

In [134]:
print(classification_report(test_Y, pred_Y))

               precision    recall  f1-score   support

     Business       0.95      0.95      0.95       562
Entertainment       0.93      0.98      0.96       553
       Health       0.93      0.76      0.83        83
   Technology       0.84      0.77      0.80       136

     accuracy                           0.93      1334
    macro avg       0.91      0.86      0.89      1334
 weighted avg       0.93      0.93      0.93      1334



正解率：93%

学習アルゴリズム: liblinear

正則化パラメータC: 100